In [120]:
from google.cloud import bigquery
import pandas as pd
import missingno as msno
import nltk
import numpy as np

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: hackathon-08


# Data Ingestion

In [143]:
query = """
SELECT
    eanIsbn,
    nome,
    categoria,
    subCategoriaNivel4,
    departamento,
    tipoProduto,
    cor,
    sabor,
    tamanho,
    skuReferencia,
    skuNetshoes,
    precoPor,
    precoDe,
    valorGenero,
    peso,
    marca,
    dimensao.altura,
    dimensao.largura,
    dimensao.profundidade,
    descricao,
    nomeGenero,
    CASE
      WHEN skuReferencia IS NOT NULL AND skuNetshoes IS NULL THEN skuReferencia
      ELSE skuNetshoes
    END as skuNetshoesReferencia
    FROM `hackathon-08.dadosbrutos.sellerProducts`
    WHERE 
    (skuReferencia IS NULL and statusProdutoLojista.statusLojista = "APROVADO" and skuNetshoes IS NOT NULL
    OR
    skuReferencia IS NOT NULL and statusProdutoLojista.statusLojista = "APROVADO" and skuNetshoes IS NULL
    OR
    skuReferencia IS NOT NULL and statusProdutoLojista.statusLojista = "APROVADO" and skuNetshoes IS NOT NULL)
    AND skuReferencia in (
      'D16-2255-118-42',
    'D16-2008-026-44',
    'D16-2905-006-43',
    'D13-2253-026-43',
    'D16-2905-006-41',
    'D16-2704-108-40',
    'D16-2905-006-42',
    'D16-2905-006-40',
    'D16-1979-086-39',
    'D22-2026-006-35',
    'D16-2952-118-42',
    'D16-2905-006-39',
    'D16-2255-118-41',
    'D16-2952-118-41',
    'D16-2249-234-38',
    'D22-1131-006-36',
    'D22-1131-006-38',
    'D22-2021-178-40',
    'D16-2704-038-43',
    'D13-2253-026-40',
    'D22-2021-178-39',
    'D22-2139-360-39',
    'D22-2139-360-40',
    'D22-2139-360-42',
    'D16-2249-234-36',
    'D16-2249-189-37',
    'D16-2249-189-39',
    'D16-1979-086-42',
    'D16-1969-180-38',
    'D16-2001-026-39',
    'COL-3966-086-42',
    'D16-2009-026-43',
    'D16-1969-180-40',
    'D16-1969-002-43',
    'D16-2008-026-39',
    'D16-1969-002-38',
    'D16-1969-180-41',
    'D16-1969-180-39',
    'COL-3966-086-43',
    'D16-2249-189-38',
    'D16-2255-118-39',
    'D16-1969-180-42',
    'D16-2249-189-35',
    'D22-1131-006-37',
    'D22-2026-006-39',
    'D16-1959-084-36',
    'D16-1969-180-43',
    'D22-2026-006-36',
    'D16-2008-026-40',
    'D16-2249-189-36',
    'D16-2001-026-41',
    'D13-2253-026-38',
    'D16-1959-084-38',
    'D18-3015-778-38',
    'D16-2952-118-40',
    'D22-2139-360-41',
    'D22-2139-360-43',
    'D16-2008-026-43',
    'D16-2008-026-42',
    'D16-2249-234-34',
    'D16-2249-234-35',
    'D16-2255-118-43',
    'D16-1979-086-43',
    'D16-1969-002-40',
    'D22-1910-006-43',
    'D16-2952-118-39',
    'D16-2952-118-43',
    'D13-2253-026-39',
    'D22-1131-006-35',
    'D16-2249-234-37',
    'D13-2253-026-42',
    'D16-2008-026-41',
    'D16-2001-026-43',
    'D22-1131-006-39',
    'D16-2704-038-41',
    'D16-2255-118-40',
    'D16-2905-006-44')
    LIMIT 50000
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()

In [144]:
df.to_csv('raw_data.csv', index=False)

In [191]:
df = pd.read_csv('raw_data.csv')

In [192]:
df.shape

(935, 22)

In [193]:
df.head(3)

,eanIsbn,nome,categoria,subCategoriaNivel4,departamento,tipoProduto,cor,sabor,tamanho,skuReferencia,...,precoDe,valorGenero,peso,marca,altura,largura,profundidade,descricao,nomeGenero,skuNetshoesReferencia
0,4.060509e+12,Chuteira Adidas Predator 19.4 TF Society Azul,Chuteiras,NaN,Futebol,Chuteiras,Azul,NaN,42,COL-3966-086-42,...,NaN,Homem,690.00,Adidas,24.0,9.0,10.0,Eleve a precisão nos chutes com a Chuteira Soc...,Gênero,COL-3966-086-42
1,4.060509e+12,Chuteira Adidas Predator 19.4 TF Society Azul,Chuteiras,NaN,Futebol,Chuteiras,Azul,NaN,42,COL-3966-086-42,...,NaN,Homem,690.00,Adidas,24.0,9.0,10.0,Eleve a precisão nos chutes com a Chuteira Soc...,Gênero,COL-3966-086-42
2,4.060509e+12,Chuteira Adidas Predator 19.4 TF Society Azul,NaN,NaN,Futebol,Chuteiras,Azul,NaN,42,COL-3966-086-42,...,NaN,Homem,0.75,Adidas,12.0,24.0,36.0,A Chuteira Adidas Society Predator 19.4 TF con...,Gênero,COL-3966-086-42


In [194]:
df['descr'] = df.apply(lambda r: f"{r['nome']} {r['descricao']} {r['eanIsbn']}", axis=1)

In [195]:
_ = df.pop('nome')
_ = df.pop('descricao')
_ = df.pop('skuReferencia')
_ = df.pop('skuNetshoes')
_ = df.pop('eanIsbn')
del _

# Data preprocessing

In [196]:
df.columns

Index(['categoria', 'subCategoriaNivel4', 'departamento', 'tipoProduto', 'cor',
       'sabor', 'tamanho', 'precoPor', 'precoDe', 'valorGenero', 'peso',
       'marca', 'altura', 'largura', 'profundidade', 'nomeGenero',
       'skuNetshoesReferencia', 'descr'],
      dtype='object')

In [197]:
from unidecode import unidecode

def normalize(s):
    if str(s) == 'nan' or type(s) == type(None):
        return ""
    return unidecode(str(s)).lower()

In [198]:
for i, col in enumerate(df.columns):
    print(i, col)

0 categoria
1 subCategoriaNivel4
2 departamento
3 tipoProduto
4 cor
5 sabor
6 tamanho
7 precoPor
8 precoDe
9 valorGenero
10 peso
11 marca
12 altura
13 largura
14 profundidade
15 nomeGenero
16 skuNetshoesReferencia
17 descr


In [199]:
text_cols = ['categoria', 'subCategoriaNivel4', 'departamento', 'tipoProduto', 'cor', 'sabor', 'tamanho', 'valorGenero', 'marca', 'nomeGenero', 'descr']

for col in text_cols:
    df[col] = df[col].apply(normalize)
    df[col] = pd.Categorical(df[col])

for col in text_cols:
    if col == 'descr':
        continue
    dummies = pd.get_dummies(df[col], prefix=col, dummy_na=True).iloc[:, :-1]
    df = pd.concat([df, dummies], axis=1)
    _ = df.pop(col)

In [200]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = nltk.corpus.stopwords.words("portuguese")

tvec = TfidfVectorizer(min_df=.1, max_df=.15, stop_words=stop_words, ngram_range=(1, 2))
tvec_weights = tvec.fit_transform(df.descr)
descr_w = pd.DataFrame(tvec_weights.todense(), columns=tvec.get_feature_names())

_ = df.pop('descr')

df = pd.concat([df, descr_w], axis=1)
df.head(10)

,precoPor,precoDe,peso,altura,largura,profundidade,skuNetshoesReferencia,categoria_,categoria_chuteiras,categoria_tenis,...,proporciona maior,resistencia,respirabilidade,sobreposicoes,society,tradicional,treinos,treinos academia,visual,wave mirai
0,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,0,1,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,0,1,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,0.75,12.0,24.0,36.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,0.837367,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,0,1,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,0.83,10.0,20.0,30.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,0.353541,0.0,0.0,0.0,0.0,0.0
5,NaN,NaN,1.00,13.0,20.0,34.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,0.813399,0.0,0.0,0.0,0.0,0.0
6,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
7,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
8,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
9,NaN,NaN,690.00,24.0,9.0,10.0,COL-3966-086-42,1,0,0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0


In [201]:
y = df.pop('skuNetshoesReferencia').values
X = df.iloc[:, :-1].values

In [202]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)

In [203]:
from sklearn.impute import SimpleImputer as Imputer
imputer = Imputer(missing_values=np.nan, strategy = 'mean')
imputer = imputer.fit(X[:, :2])
X[:, :2] = imputer.transform(X[:, :2])

In [204]:
df.columns

Index(['precoPor', 'precoDe', 'peso', 'altura', 'largura', 'profundidade',
       'categoria_', 'categoria_chuteiras', 'categoria_tenis',
       'categoria_tenis performance',
       ...
       'proporciona maior', 'resistencia', 'respirabilidade', 'sobreposicoes',
       'society', 'tradicional', 'treinos', 'treinos academia', 'visual',
       'wave mirai'],
      dtype='object', length=108)

In [205]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=.33)

In [213]:
from sklearn.neighbors import KNeighborsClassifier as knn

classifier = knn(n_neighbors=1)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [214]:
y_pred = classifier.predict(X_test)

In [208]:
from sklearn.metrics import accuracy_score as acs
acs(y_test, y_pred)

0.7152103559870551